In [1]:
from enum import Enum
from itertools import product
import logging
from logging import Logger
import random

import pandas as pd
import numpy as np
from tabulate import tabulate
from termcolor import colored
from tqdm import tqdm


from word_grid import WordGrid, Direction



MIN_WORD_LEN = 3

In [3]:

def get_logger():
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.ERROR)
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.ERROR)
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)
    return logger

logger = get_logger()

In [19]:
puzzle = WordGrid((5,10))
puzzle

-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -

In [26]:
word_index = pd.read_csv("data/word_index.csv", encoding='utf-8')
dictionary = word_index[word_index["lang_code"] == "en"]
dictionary["word"] = dictionary["word"].astype(str)
dictionary = dictionary[dictionary["len"] >= MIN_WORD_LEN]
dictionary = dictionary[dictionary["len"] <= max(puzzle.shape)]
dictionary = dictionary[~dictionary["word"].str.contains(r"[0-9-]")]
print(len(dictionary))

C:\Users\charl\AppData\Local\Temp\ipykernel_2620\4029767802.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dictionary["word"] = dictionary["word"].astype(str)


704183


In [20]:
puzzle.reset()
while not puzzle.add_word((1,2), Direction.ACROSS, dictionary["word"].sample(1).item()):
    pass
print(puzzle)

-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -
-  p  o  l  y  s  p  e  r  m
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  -  -  -


In [21]:
def get_candidates(puzzle: WordGrid, position: tuple, direction: Direction, blacklist: list):

    candidates = dictionary[dictionary["word"].apply(lambda w: puzzle.validate_word(position, direction, w))]
    candidates = candidates[~candidates["word"].isin(blacklist)]
        
    return candidates
    

In [22]:
n = 0
direction = Direction.DOWN
word_list = []
puzzle.reset()
positions = {
    Direction.DOWN: {pos: [] for pos in product(range(puzzle.shape[1]), range(puzzle.shape[0] - MIN_WORD_LEN))},
    Direction.ACROSS: {pos: [] for pos in product(range(puzzle.shape[1] - MIN_WORD_LEN), range(puzzle.shape[0]))}
}
random.seed(12)
pbar = tqdm()
while n < 12:
    if len(positions[direction]) == 0:
        if len(positions[Direction.flip(direction)]) == 0:
            break
        direction = Direction.flip(direction)
    
    position = random.choice(list(positions[direction]))
    
    blacklist = positions[direction][position] + word_list
    candidates = get_candidates(puzzle, position, direction, blacklist)

    if len(candidates) == 0:
        positions[direction].pop(position, None)
        continue
    
    try:
        word = candidates["word"].sample(1, weights=np.log(candidates.freq)).item()
    except:
        word = candidates["word"].sample(1).item()
    
    pbar.update(n)
    #pbar.set_description(f"word: {word}, pos: {position}, dir: {direction.name.lower()}, cnd: {len(candidates)}, slots {len(positions[Direction.DOWN])}", refresh=True)
    print(f"word: {word}, pos: {position}, dir: {direction.name.lower()}, cnd: {len(candidates)}, slots {len(positions[Direction.DOWN])}d {len(positions[Direction.ACROSS])}a")
    
    if puzzle.add_word(position, direction, word):
        if len(positions[Direction.flip(direction)]) > 0:
            direction = Direction.flip(direction)

        positions[direction].pop(position, None)    
        word_list.append(word)
        n += 1
        print(puzzle)
    else:
        positions[direction][position].append(word)
        logger.info(f"Can't place word {word} at {position}")

print(word_list)
print(puzzle)

1it [00:37, 37.97s/it]


word: Pope, pos: (7, 1), dir: down, cnd: 47791, slots 20d 35a
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  p  -  -
-  -  -  -  -  -  -  o  -  -
-  -  -  -  -  -  -  p  -  -
-  -  -  -  -  -  -  e  -  -


word: try out, pos: (3, 2), dir: across, cnd: 32425, slots 20d 35a
-  -  -  -  -  -  -  -  -  -
-  -  -  -  -  -  -  p  -  -
-  -  -  t  r  y     o  u  t
-  -  -  -  -  -  -  p  -  -
-  -  -  -  -  -  -  e  -  -


word: haul, pos: (8, 0), dir: down, cnd: 3260, slots 20d 35a
-  -  -  -  -  -  -  -  h  -
-  -  -  -  -  -  -  p  a  -
-  -  -  t  r  y     o  u  t
-  -  -  -  -  -  -  p  l  -
-  -  -  -  -  -  -  e  -  -


word: narrate, pos: (1, 4), dir: across, cnd: 149666, slots 20d 34a
-  -  -  -  -  -  -  -  h  -
-  -  -  -  -  -  -  p  a  -
-  -  -  t  r  y     o  u  t
-  -  -  -  -  -  -  p  l  -
-  n  a  r  r  a  t  e  -  -


...

word: opal, pos: (6, 1), dir: across, cnd: 2, slots 17d 30a
g  o  m  a  -  -  d  -  h  -
-  -  -  -  b  -  o  p  a  l
-  -  -  t  r  y     o  u  t
-  -  -  -  r  -  i  p  l  -
-  n  a  r  r  a  t  e  -  -


word: say, pos: (5, 0), dir: down, cnd: 399, slots 15d 30a
g  o  m  a  -  s  d  -  h  -
-  -  -  -  b  a  o  p  a  l
-  -  -  t  r  y     o  u  t
-  -  -  -  r  -  i  p  l  -
-  n  a  r  r  a  t  e  -  -


word: her, pos: (2, 3), dir: across, cnd: 297, slots 15d 24a
g  o  m  a  -  s  d  -  h  -
-  -  -  -  b  a  o  p  a  l
-  -  -  t  r  y     o  u  t
-  -  h  e  r  -  i  p  l  -
-  n  a  r  r  a  t  e  -  -


word: Isan, pos: (1, 1), dir: down, cnd: 1261, slots 15d 24a
g  o  m  a  -  s  d  -  h  -
-  i  -  -  b  a  o  p  a  l
-  s  -  t  r  y     o  u  t
-  a  h  e  r  -  i  p  l  -
-  n  a  r  r  a  t  e  -  -


word: fit, pos: (0, 1), dir: across, cnd: 816, slots 15d 11a
g  o  m  a  -  s  d  -  h  -
f  i  t  -  b  a  o  p  a  l
-  s  -  t  r  y     o  u  t
-  a  h  e  r  -  i  p  l  -
-  n  a  r  r  a  t  e  -  -
['Pope', 'try out', 'haul', 'narrate', 'do it', 'Goma', 'brrr', 'opal', 'say', 'her', 'Isan', 'fit']
g  o  m  a  -  s  d  -  h  -
f  i  t  -  b  a  o  p  a  l
-  s  -  t  r  y     o  u  t
-  a  h  e  r  -  i  p  l  -
-  n  a  r  r  a  t  e  -  -


In [15]:
def custom_print(puzzle: WordGrid):
    to_print = []
    for chars, states in zip(puzzle.puzzle, puzzle.state):
        data = []
        for char, state in zip(chars, states):
            if state & Direction.ACROSS.value and state & Direction.DOWN.value:
                color = "magenta"
            elif state & Direction.ACROSS.value:
                color = "blue"
            elif state & Direction.DOWN.value:
                color = "yellow"
            else:
                color = "white"
            data.append(colored(char, color))
        to_print.append(data)
    print(tabulate(to_print))

In [25]:
custom_print(puzzle)

-  -  -  -  -  -  -  -  -  -
g  o  m  a  -  s  d  -  h  -
f  i  t  -  b  a  o  p  a  l
-  s  -  t  r  y     o  u  t
-  a  h  e  r  -  i  p  l  -
-  n  a  r  r  a  t  e  -  -
-  -  -  -  -  -  -  -  -  -
